In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

In [2]:
labels = ['Cover', 'JMiPOD', 'JUNIWARD', 'UERD']
img_size = 128
batch_size = 500
data_dir = r'D:\Repo_Licenta\resized_grayscale_alaska2'

In [3]:
# labels = ['Cover', 'JMiPOD', 'JUNIWARD', 'UERD']
# img_size = 256
# x_train = []
# y_train = []
#
#
# def get_data(data_dir):
#     for label in labels:
#         path = os.path.join(data_dir, label)
#         class_num = labels.index(label)
#         index = 0
#         for img in os.listdir(path):
#             if index == 100:
#                 break
#             try:
#                 img_arr = cv2.imread(os.path.join(path, img))  #[..., ::-1]  #convert BGR to RGB format
#                 gray_img = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
#                 resized_arr = cv2.resize(gray_img, (img_size, img_size))  # Reshaping images to preferred size
#                 x_train.append(resized_arr)
#                 y_train.append(class_num)
#             except Exception as e:
#                 print(e)
#             index += 1

In [4]:
x_test = []


def get_test_data(path):
    for img in os.listdir(path):
        try:
            img_arr = cv2.imread(os.path.join(path, img))  #[..., ::-1]  #convert BGR to RGB format
            gray_img = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
            resized_arr = cv2.resize(gray_img, (img_size, img_size))  # Reshaping images to preferred size
            x_test.append(resized_arr)
        except Exception as e:
            print(e)

In [5]:
#get_data(r'D:\Repo_Licenta\alaska2-steganalysis')
#x_train
#y_train

In [6]:
#get_test_data(r'D:\Repo_Licenta\alaska2-steganalysis\Test')
#x_test

In [7]:
# Normalize the data
# x_train = np.array(x_train) / 255
# x_test = np.array(x_test) / 255
#
# x_train.reshape(-1, img_size, img_size, 1)
#
# x_test.reshape(-1, img_size, img_size, 1)


In [8]:
# Define the model

model = models.Sequential()
model.add(layers.Conv2D(32, 3, padding="same", activation="relu", input_shape=(img_size, img_size, 1)))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(32, 3, padding="same", activation="relu"))
model.add(layers.MaxPool2D())

model.add(layers.Conv2D(64, 3, padding="same", activation="relu"))
model.add(layers.MaxPool2D())
model.add(layers.Dropout(0.4))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(4, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0

In [9]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]

(None, 128, 128, 1) <dtype: 'float32'>
(None, 4) <dtype: 'float32'>


[None]

In [11]:
for batch in range(5000 // batch_size):
    print("Batch:", batch)
    x_train = []
    y_train = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for index in range(batch * batch_size, (batch + 1) * batch_size):
            try:
                #img = os.listdir(path)[index]
                img_arr = cv2.imread(os.path.join(path, os.listdir(path)[index]), cv2.IMREAD_GRAYSCALE)
                x_train.append(img_arr)
                y_train.append(class_num)
            except Exception as e:
                print(e)
    x_train = np.array(x_train) / 255
    x_train = x_train[..., np.newaxis]
    x_train.reshape(-1, img_size, img_size, 1)
    y_train = np.array(y_train)
    p = np.random.permutation(len(x_train))
    x_train = x_train[p]
    y_train = y_train[p]
    history = model.fit(x_train, y_train, epochs=3, verbose=1)
    # acc = history.history['accuracy']
    # loss = history.history['loss']
    #
    # epochs_range = range(3)
    #
    # plt.figure(figsize=(15, 15))
    # plt.subplot(2, 1, 1)
    # plt.plot(epochs_range, acc, label='Training Accuracy')
    # plt.legend(loc='lower right')
    # plt.title('Training Accuracy')
    #
    # plt.subplot(2, 1, 2)
    # plt.plot(epochs_range, loss, label='Training Loss')
    # plt.legend(loc='upper right')
    # plt.title('Training Loss')
    # plt.show()


Batch: 0
Epoch 1/3


C:\Users\nicu2\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\backend.py:4929: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


63/63 [==============================] - 11s 67ms/step - loss: 1.3887 - accuracy: 0.2430